In [6]:
#Strategy 2 with legendre comparison
import math
import random
import numpy as np
import timeit
from scipy.stats import norm

#from decimal import *

#getcontext().prec=28
NumSim = 10000

lambda1 = 8
r = 0.05
sigma = 0.3
sigmaA = 0.05
muA = 0
R = 1
X0 = 55
S0 = 50
H = 45
TT = 1
X = 55
logH = np.log(H)
tau = 0
gamma = 0

k = math.exp(muA + .5*sigmaA**2)-1

c = r-.5*sigma**2-lambda1*k

#original strategy 2
DiscPayoff = np.zeros(NumSim)
#strategy 2 use legendre gaussian integration rather than Laplace transformation
DiscPayoff2 = np.zeros(NumSim)
#range estimated by using end points of each interjump, thus giving lower and upper bounds
DiscPayoffl = np.zeros(NumSim)
DiscPayoffr = np.zeros(NumSim)
# K-1 is the number of jumps 
K = 0

start = timeit.default_timer()

a = np.polynomial.legendre.leggauss(50)[0]
u = np.zeros(50)
w = np.polynomial.legendre.leggauss(50)[1]
g = np.zeros(50)

for n in range(NumSim):
    
    #interjump time
    T = np.zeros(2*lambda1)
       
    
    fitflag = True
    
    i = 1
    while fitflag:
        
        temp = -math.log(1.0 - random.random()) /lambda1      
        Temp = min(T[i-1]+temp,1)
            
    #in case goes beyond T=1
        if Temp ==1:
            K = i
            if K < 2*lambda1:
                T[i] = 1
            else: T = np.append(T,1)
            fitflag = False
        elif i>=2*lambda1: 
            T = np.append(T, Temp)
            i = i+1
        else:
            T[i] = Temp
            i = i + 1  
  
            
    T = T*TT  
   
        
    #3
    #asset value @ jump
    X_minus = np.zeros(K+1)
    X_plus = np.zeros(K+1)
    #jump size
    A=np.zeros(K+1)

    X_plus[0]=np.log(S0)

    #at each jump point
    for i in range(1, K):
        
        X_minus[i] = np.random.normal(X_plus[i-1]+c*(T[i]-T[i-1]), sigma*math.sqrt(T[i]-T[i-1]))
        A[i]=np.random.normal(muA, sigmaA)
        X_plus[i] = X_minus[i]+A[i]
    
    X_minus[K]= np.random.normal(X_plus[K-1]+c*(TT-T[K-1]), sigma*math.sqrt(TT-T[K-1]))
    X_plus[K]=X_minus[K]

    #4
    P = np.zeros(K+1)
# never used, technicality
    P[0] = 1
    
    i = 1
    
    contflag = True

    while contflag:
        
        if i > K:
            DiscPayoff[n] = DiscPayoff[n]+math.exp(-r*TT)*max(math.exp(X_minus[K])-X,0)*np.prod(P)
            DiscPayoff2[n] = DiscPayoff2[n]+math.exp(-r*TT)*max(math.exp(X_minus[K])-X,0)*np.prod(P)
#            DiscPayoff2[n] = R*math.exp(-r*TT)*max(math.exp(XT)-X,0)
            DiscPayoffl[n] = DiscPayoffl[n]+math.exp(-r*TT)*max(math.exp(X_minus[K])-X,0)*np.prod(P)
            DiscPayoffr[n] = DiscPayoffr[n]+math.exp(-r*TT)*max(math.exp(X_minus[K])-X,0)*np.prod(P)
#            print(DiscPayoff[n], "at the end")
            contflag = False
        else: 
#            print(i)
#            print('time value is', T[i-1], 'to', T[i])
#            print('underlying value is', math.exp(X_plus[i-1]), 'to', math.exp(X_minus[i]))
              
            tau = T[i] - T[i-1]
            
            u = 0.5*(T[i]+T[i-1]+tau*a)
            B = 8 - 2*r*tau+2*r/sigma**2*(X_plus[i-1]-X_minus[i])*(X_plus[i-1]+X_minus[i]-2*logH)
    
            if X_minus[i] <= logH:        
                AA = 2*r*tau/sigma*(X_plus[i-1]+X_minus[i]-2*logH)
                CC = math.sqrt(2*math.pi*tau)*math.exp((X_plus[i-1]-X_minus[i])**2/2*tau*sigma**2)*(norm.cdf((X_plus[i-1]-X_minus[i])/math.sqrt(2*tau*sigma**2))-1)
                J = math.exp(-r*T[i-1])+r*(X_plus[i-1]-logH)*math.exp(r*T[i]-2*r*T[i-1])/(8*sigma)*(AA+CC*B)
                DiscPayoff[n] = DiscPayoff[n]+R*J*np.prod(P[0:i])
                gamma = 1.0/(math.sqrt(2*math.pi*tau)*sigma)*math.exp(-(X_plus[i-1]-X_minus[i]+c*tau)**2/(2*sigma**2*tau))
                for j in range(50):
                    #u[j] = (T[j]+a[j]*T[j-1])/(a[j]+1)
                    g[j] = math.exp(-r*u[j])*(X_plus[i-1]-logH)/(2*math.pi*gamma*sigma**2)*math.pow((u[j]-T[i-1]),-1.5)*math.pow((T[i]-u[j]),-0.5)*math.exp(-(((X_minus[i]-logH-c*(T[i]-u[j]))**2/(2*(T[i]-u[j])*sigma**2))+((X_plus[i-1]-logH-c*(u[j]-T[i-1]))**2/(2*(u[j]-T[i-1])*sigma**2))))
	#                 
                DiscPayoff2[n] = DiscPayoff2[n] + R*np.prod(P[0:i])*np.dot(w, g)*tau/2
                DiscPayoffl[n] = DiscPayoffl[n] + R*np.prod(P[0:i])*math.exp(-r*T[i-1])
                DiscPayoffr[n] = DiscPayoffr[n] + R*np.prod(P[0:i])*math.exp(-r*T[i])
#                print(DiscPayoff[n], "interjump at interval", i)
#                print(DiscPayoff2[n], "interjump at interval", i)
                contflag = False
            else: 
                P[i] = 1 - math.exp(-2*(logH-X_plus[i-1])*(logH-X_minus[i])/(tau*sigma**2))
                AA = 2*r*tau/sigma*(X_plus[i-1]-X_minus[i])*(1-P[i])
                CC = math.sqrt(2*math.pi*tau)*math.exp((X_plus[i-1]-X_minus[i])**2/2*tau*sigma**2)*(norm.cdf((X_plus[i-1]+X_minus[i]-2*logH)/math.sqrt(2*tau*sigma**2))-1)
                J = math.exp(-r*T[i-1])*(1-P[i])+r*(X_plus[i-1]-logH)*math.exp(r*T[i]-2*r*T[i-1])/(8*sigma)*(AA+CC*B)
                DiscPayoff[n] = DiscPayoff[n]+R*J*np.prod(P[0:i])
                gamma = 1.0/(math.sqrt(2*math.pi*tau)*sigma)*math.exp(-(X_plus[i-1]-X_minus[i]+c*tau)**2/(2*sigma**2*tau))
                for j in range(50):
                    #u[j] = (T[j]+a[j]*T[j-1])/(a[j]+1)
                    g[j] = math.exp(-r*u[j])*(X_plus[i-1]-logH)/(2*math.pi*gamma*sigma**2)*math.pow((u[j]-T[i-1]),-1.5)*math.pow((T[i]-u[j]),-0.5)*math.exp(-(((X_minus[i]-logH-c*(T[i]-u[j]))**2/(2*(T[i]-u[j])*sigma**2))+((X_plus[i-1]-logH-c*(u[j]-T[i-1]))**2/(2*(u[j]-T[i-1])*sigma**2))))
	#                 
                DiscPayoff2[n] = DiscPayoff2[n] + R*np.prod(P[0:i])*np.dot(w, g)*tau/2
                DiscPayoffl[n] = DiscPayoffl[n] + R*np.prod(P[0:i])*math.exp(-r*T[i-1])*(1-P[i])
                DiscPayoffr[n] = DiscPayoffr[n] + R*np.prod(P[0:i])*math.exp(-r*T[i])*(1-P[i])
#                print(J, P[i])
                if X_plus[i]<= logH and i<K:
                    DiscPayoff[n] = DiscPayoff[n]+R*math.exp(-r*T[i])*np.prod(P[0:i+1])
                    DiscPayoff2[n] = DiscPayoff2[n]+R*math.exp(-r*T[i])*np.prod(P[0:i+1])
                    DiscPayoffl[n] = DiscPayoffl[n]+R*math.exp(-r*T[i])*np.prod(P[0:i+1])
                    DiscPayoffr[n] = DiscPayoffr[n]+R*math.exp(-r*T[i])*np.prod(P[0:i+1])
#					DiscPayoffl[n] = R*math.exp(-r*T[i])
#					DiscPayoffr[n] = R*math.exp(-r*T[i])
#                    print(DiscPayoff[n], "jump at point", i)
#                    print(DiscPayoff2[n], "jump at point", i)
                    contflag = False
                else: i = i + 1

print("original strategy 2 and its standard deviation")
print (np.average(DiscPayoff), np.std(DiscPayoff,ddof=1))
print("legendre modified strategy 2 and its standard deviation")
print (np.average(DiscPayoff2), np.std(DiscPayoff2,ddof=1))
print("range", "(", np.average(DiscPayoffr), np.average(DiscPayoffl), ")")
#end = timeit.default_timer()
#print(end - start)   



original strategy 2 and its standard deviation
4.62942548919 9.53008660939
legendre modified strategy 2 and its standard deviation
4.62144803409 9.53300877582
range ( 4.6261071948 4.63276478749 )
38.34472470298857


In [ ]:
# Conclusion, original s2 works better with lower SD while the proposed Legendre method falls out of the range frequently

In [34]:
#Under same path for T and X, compare the performance of Strategy 1, proposed stragy 1 improved by
#using stragy 2 method to evaluate the expect value instead of just bg(s) in part 4, as well as range given
#by evaluating at endpoint during interjump, strategy 2
# In[138]:

#1
import math
import random
import numpy as np
import timeit
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd

#from decimal import *

#getcontext().prec=28
NumSim = 100000

lambda1 = 8
r = 0.05
sigma = 0.3
sigmaA = 0.05
muA = 0
R = 1
X0 = 55
S0 = 50
H = 45
TT = 1
X = 55
logH = np.log(H)
tau = 0
gamma = 0

k = math.exp(muA + .5*sigmaA**2)-1

c = r-.5*sigma**2-lambda1*k

DiscPayoff = np.zeros(NumSim)
DiscPayoff1 = np.zeros(NumSim)
DiscPayoff2 = np.zeros(NumSim)
DiscPayoffl = np.zeros(NumSim)
DiscPayoffr = np.zeros(NumSim)
# K is the number of jumps
K = 0
#a = np.zeros(20)
#d = np.zeros(20)
start = timeit.default_timer()

#Store result
d={'Strategy 1': np.zeros(5), 'S1 Sd': np.zeros(5), 'Strategy 2': np.zeros(5), 'S2 Sd': np.zeros(5), 'Strategy 1 combined with 2': np.zeros(5),'S12 Sd': np.zeros(5), 'lower bound': np.zeros(5), 'upper bound': np.zeros(5),}
columns=['Strategy 1', 'S1 Sd', 'Strategy 2', 'S2 Sd', 'Strategy 1 combined with 2','S12 Sd', 'lower bound', 'upper bound']
df=pd.DataFrame(data=d, columns=columns)

for j in range(5):
    
    DiscPayoff[...] = 0
    DiscPayoff2[...] = 0
    DiscPayoffl[...] = 0
    DiscPayoffr[...] = 0
    for n in range(NumSim):
    
        #interjump time
        T = np.zeros(2*lambda1)

        fitflag = True

        i = 1
        while fitflag:
            temp = -math.log(1.0 - random.random()) /lambda1      
            Temp = min(T[i-1]+temp,1)
            #in case goes beyond T=1
            if Temp ==1:
                K = i
                if K < 2*lambda1:
                    T[i] = 1
                else: T = np.append(T,1)
                fitflag = False
            elif i>=2*lambda1: 
                T = np.append(T, Temp)
                i = i+1
            else:
                T[i] = Temp
                i = i + 1  
        
        T = T*TT  
		
		#3
		#asset value @ jump
        X_minus = np.zeros(K+1)
        X_plus = np.zeros(K+1)
		#jump size
        A=np.zeros(K+1)
        
        X_plus[0]=np.log(S0)
        
		#at each jump point
        for i in range(1, K):
			
            X_minus[i] = np.random.normal(X_plus[i-1]+c*(T[i]-T[i-1]), sigma*math.sqrt(T[i]-T[i-1]))
            A[i]=np.random.normal(muA, sigmaA)
            X_plus[i] = X_minus[i]+A[i]
		
        X_minus[K]= np.random.normal(X_plus[K-1]+c*(TT-T[K-1]), sigma*math.sqrt(TT-T[K-1]))
        X_plus[K]= X_minus[K]
		#4
        P = np.zeros(K+1)
		
        i = 1
		
	#    print(K)
		
	#    print(np.exp(X_plus))
		
	#    print(np.exp(X_minus))
		
        contflag = True

        while contflag:
			
            if i == K+1:
				
                DiscPayoff[n] = R*math.exp(-r*TT)*max(math.exp(X_minus[K])-X,0)
                DiscPayoff1[n] = R*math.exp(-r*TT)*max(math.exp(X_minus[K])-X,0)
                DiscPayoffl[n] = R*math.exp(-r*TT)*max(math.exp(X_minus[K])-X,0)
                DiscPayoffr[n] = R*math.exp(-r*TT)*max(math.exp(X_minus[K])-X,0)
	#            print(XT, DiscPayoff[n], "at", TT)
                contflag = False
            else: 
	#            print(i)
			
	#            print('time value is', T[i-1], 'to', T[i])
		
	#            print('underlying value is', math.exp(X_plus[i-1]), 'to', math.exp(X_minus[i]))
			
	#            print('knock out is', H)
			
                tau = T[i] - T[i-1]
		
                if X_minus[i] > logH:        
                    P[i] = 1 - math.exp(-2*(logH-X_plus[i-1])*(logH-X_minus[i])/(tau*sigma**2))
	#            print(P[i])   
    
                if P[i] < 1: 
                    b = tau/(1-P[i])                      
                    s = np.random.uniform(T[i-1], T[i-1]+b)
	#                print('Pi is', P[i], 'and s is', s, 'and b is', b)
			#   cross at s
                    if s <= T[i]:    
                        gamma = 1.0/(math.sqrt(2*math.pi*tau)*sigma)*math.exp(-(X_plus[i-1]-X_minus[i]+c*tau)**2/(2*sigma**2*tau))
                        g = (X_plus[i-1]-logH)/(2*math.pi*gamma*sigma**2)*math.pow((s-T[i-1]),-1.5)*math.pow((T[i]-s),-0.5)*math.exp(-(((X_minus[i]-logH-c*(T[i]-s))**2/(2*(T[i]-s)*sigma**2))+((X_plus[i-1]-logH-c*(s-T[i-1]))**2/(2*(s-T[i-1])*sigma**2))))
	#                    print(gamma, g, b*g)
                        DiscPayoff[n] = R*b*math.exp(-r*s)*g
                        DiscPayoffl[n] = R*math.exp(-r*T[i-1])
                        DiscPayoffr[n] = R*math.exp(-r*T[i])
                        B = 8 - 2*r*tau+2*r/sigma**2*(X_plus[i-1]-X_minus[i])*(X_plus[i-1]+X_minus[i]-2*logH)
    
                        if X_minus[i] <= logH:        
                            AA = 2*r*tau/sigma*(X_plus[i-1]+X_minus[i]-2*logH)
                            CC = math.sqrt(2*math.pi*tau)*math.exp((X_plus[i-1]-X_minus[i])**2/2*tau*sigma**2)*(norm.cdf((X_plus[i-1]-X_minus[i])/math.sqrt(2*tau*sigma**2))-1)
                            J = math.exp(-r*T[i-1])+r*(X_plus[i-1]-logH)*math.exp(r*T[i]-2*r*T[i-1])/(8*sigma)*(AA+CC*B)
                        else: 
                            P[i] = 1 - math.exp(-2*(logH-X_plus[i-1])*(logH-X_minus[i])/(tau*sigma**2))
                            AA = 2*r*tau/sigma*(X_plus[i-1]-X_minus[i])*(1-P[i])
                            CC = math.sqrt(2*math.pi*tau)*math.exp((X_plus[i-1]-X_minus[i])**2/2*tau*sigma**2)*(norm.cdf((X_plus[i-1]+X_minus[i]-2*logH)/math.sqrt(2*tau*sigma**2))-1)
                            J = math.exp(-r*T[i-1])*(1-P[i])+r*(X_plus[i-1]-logH)*math.exp(r*T[i]-2*r*T[i-1])/(8*sigma)*(AA+CC*B)
                        DiscPayoff1[n] = R*J/(1-P[i])
                        contflag = False
                        break
					
                if X_plus[i]<= logH and i<K+1:
                    DiscPayoff[n] = R*math.exp(-r*T[i])
                    DiscPayoff1[n] = R*math.exp(-r*T[i])
                    DiscPayoffl[n] = R*math.exp(-r*T[i])
                    DiscPayoffr[n] = R*math.exp(-r*T[i])
	#                print(DiscPayoff[n], "jump at point", i)
                    contflag = False
                else: i = i + 1
        
        P[...] =0 
        
        P[0] = 1
    
        i = 1
    
        contflag = True

        while contflag:
        
            if i > K:
                DiscPayoff2[n] = DiscPayoff2[n]+math.exp(-r*TT)*max(math.exp(X_minus[K])-X,0)*np.prod(P)
#            print(DiscPayoff[n], "at the end")
                contflag = False
            else: 
##            print(i)
##            print('time value is', T[i-1], 'to', T[i])
    
##            print('underlying value is', math.exp(X_plus[i-1]), 'to', math.exp(X_minus[i]))
              
                tau = T[i] - T[i-1]
                B = 8 - 2*r*tau+2*r/sigma**2*(X_plus[i-1]-X_minus[i])*(X_plus[i-1]+X_minus[i]-2*logH)
    
                if X_minus[i] <= logH:        
                    AA = 2*r*tau/sigma*(X_plus[i-1]+X_minus[i]-2*logH)
                    CC = math.sqrt(2*math.pi*tau)*math.exp((X_plus[i-1]-X_minus[i])**2/2*tau*sigma**2)*(norm.cdf((X_plus[i-1]-X_minus[i])/math.sqrt(2*tau*sigma**2))-1)
                    J = math.exp(-r*T[i-1])+r*(X_plus[i-1]-logH)*math.exp(r*T[i]-2*r*T[i-1])/(8*sigma)*(AA+CC*B)
                    DiscPayoff2[n] = DiscPayoff2[n]+R*J*np.prod(P[0:i])
#                print(DiscPayoff[n], "interjump at interval", i)
                    contflag = False
                else: 
                    P[i] = 1 - math.exp(-2*(logH-X_plus[i-1])*(logH-X_minus[i])/(tau*sigma**2))
                    AA = 2*r*tau/sigma*(X_plus[i-1]-X_minus[i])*(1-P[i])
                    CC = math.sqrt(2*math.pi*tau)*math.exp((X_plus[i-1]-X_minus[i])**2/2*tau*sigma**2)*(norm.cdf((X_plus[i-1]+X_minus[i]-2*logH)/math.sqrt(2*tau*sigma**2))-1)
                    J = math.exp(-r*T[i-1])*(1-P[i])+r*(X_plus[i-1]-logH)*math.exp(r*T[i]-2*r*T[i-1])/(8*sigma)*(AA+CC*B)
                    DiscPayoff2[n] = DiscPayoff2[n]+R*J*np.prod(P[0:i])
#                print(J, P[i])
                    if X_plus[i]<= logH and i<K:
                        DiscPayoff2[n] = DiscPayoff2[n]+R*math.exp(-r*T[i])*np.prod(P[0:i+1])
##                    print(DiscPayoff[n], "jump at point", i)
                        contflag = False
                    else: i = i + 1

    df.iloc[j,0]=np.average(DiscPayoff)
    df.iloc[j,1]=np.std(DiscPayoff, ddof=1)
    df.iloc[j,2]=np.average(DiscPayoff2)
    df.iloc[j,3]=np.std(DiscPayoff2, ddof=1)
    df.iloc[j,4]=np.average(DiscPayoff1)
    df.iloc[j,5]=np.std(DiscPayoff1, ddof=1)
    df.iloc[j,6]=np.average(DiscPayoffr)
    df.iloc[j,7]=np.average(DiscPayoffl)


end = timeit.default_timer()
print(end - start)        


90.75522634361914


In [35]:
#Conclusion: proposed strategy S12 works better with lower sd and around the middle of the range while s1 falls out
#around 80% of the time. 
df


,Strategy 1,S1 Sd,Strategy 2,S2 Sd,Strategy 1 combined with 2,S12 Sd,lower bound,upper bound
0,4.483244,10.330434,4.486214,9.273352,4.482171,9.792199,4.478852,4.485555
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
